In [4]:
import ee
import geemap as gmp
import os
print ("OK!")

OK!


In [5]:
ee.Authenticate()

Enter verification code: 4/1AfJohXnLGeXYBcYFKudT47gCabR1x97QYNuS9Aa7Fx8icWO3TXhrksFWyzo

Successfully saved authorization token.


In [7]:
ee.Initialize()
print ("OK!")

OK!


In [129]:
map_1 = gmp.Map(zoom = 6)
Hexi_shp = ee.FeatureCollection("users/LST_SG/HeXi")
vis_param1 = {"out_linecolor" : "red", "outline" : 0.5, "fill_color" : "null"}

In [1]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2022-06-20', '2022-08-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

map_1.add_layer(dataset.mean(), vis_param2)
map_1.add_layer(Hexi_shp, vis_param1, name = 'Hexi')
map_1

NameError: name 'ee' is not defined